Napredna uporaba Scikit-learn

###### Branje datoteke

In [0]:
import pandas as pd

df_osnovni = pd.read_csv("drive/My Drive/FERI/MAG/TRIR/Data/happiness_oecd.csv", sep=';', decimal=',')
df_osnovni.shape
for col in df_osnovni.columns:
  if col != "Region" and col != "Country":
    df_osnovni[col] = pd.to_numeric(df_osnovni[col])
df_osnovni.dtypes

Country                                                               object
Happiness.Rank                                                       float64
Happiness.Score                                                      float64
Economy..GDP.per.Capita.                                             float64
Family                                                               float64
Health..Life.Expectancy.                                             float64
Freedom                                                              float64
Generosity                                                           float64
Trust..Government.Corruption.                                        float64
Dystopia.Residual                                                    float64
Region                                                                object
Happiness Rank 2016                                                  float64
Happiness Score 2016                                                 float64

###### Preverjanje manjkajocih vrednosti

In [0]:
df_osnovni.isna().sum()
'''
 => zapis povprecja za numericne manjkajoce vrednosti
df.fillna(df.mean(), inplace=True)
df.isnull().sum()
'''

'\n => zapis povprecja za numericne manjkajoce vrednosti\ndf.fillna(df.mean(), inplace=True)\ndf.isnull().sum()\n'

###### Zapis le numericnih stolpcev

In [0]:
df_numeric = df_osnovni.drop(['Region', 'Country'], axis=1)
df_numeric.head()

,Happiness.Rank,Happiness.Score,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,Happiness Rank 2016,Happiness Score 2016,Happiness Rank 2015,Happiness Score 2015,Air pollution (Micrograms per cubic metre),Dwellings without basic facilities (Percentage),Educational attainment (Percentage),Employees working very long hours (Percentage),Employment rate (Percentage),Feeling safe walking alone at night (Percentage),Homicide rate (Ratio),Household net adjusted disposable income (US Dollar),Household net financial wealth (US Dollar),Housing expenditure (Percentage),Labour market insecurity (Percentage),Life expectancy (Years),Life satisfaction (Average score),Long-term unemployment rate (Percentage),Personal earnings (US Dollar),Quality of support network (Percentage),Rooms per person (Ratio),Self-reported health (Percentage),Stakeholder engagement for developing regulations (Average score),Student skills (Average score),Time devoted to leisure and personal care (Hours),Voter turnout (Percentage),Water quality (Percentage),Years in education (Years),RankDiff2017,RankDiff2016
0,141.0,3.794,0.401477,0.581543,0.180747,0.106180,0.311871,0.061158,2.150801,154.0,3.360,153.0,3.575,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,-13.0,1.0
1,109.0,4.644,0.996193,0.803685,0.731160,0.381499,0.201313,0.039864,1.490442,109.0,4.655,95.0,4.959,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,0.0,14.0
2,53.0,5.872,1.091864,1.146217,0.617585,0.233336,0.069437,0.146096,2.567604,38.0,6.355,68.0,5.605,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,15.0,-30.0
3,140.0,3.795,0.858428,1.104412,0.049869,0.000000,0.097926,0.069720,1.614482,141.0,3.866,137.0,4.033,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,-1.0,4.0
4,24.0,6.599,1.185295,1.440451,0.695137,0.494519,0.109457,0.059740,2.614005,26.0,6.650,30.0,6.574,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,-2.0,-4.0


In [0]:
num_columns = df_numeric.columns
num_columns

Index(['Happiness.Rank', 'Happiness.Score', 'Economy..GDP.per.Capita.',
       'Family', 'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual',
       'Happiness Rank 2016', 'Happiness Score 2016', 'Happiness Rank 2015',
       'Happiness Score 2015', 'Air pollution (Micrograms per cubic metre)',
       'Dwellings without basic facilities (Percentage)',
       'Educational attainment (Percentage)',
       'Employees working very long hours (Percentage)',
       'Employment rate (Percentage)',
       'Feeling safe walking alone at night (Percentage)',
       'Homicide rate (Ratio)',
       'Household net adjusted disposable income (US Dollar)',
       'Household net financial wealth (US Dollar)',
       'Housing expenditure (Percentage)',
       'Labour market insecurity (Percentage)', 'Life expectancy (Years)',
       'Life satisfaction (Average score)',
       'Long-term unemployment rate (Percentage)',
       'Personal earnin

###### Standardiziranje stevilskih atributov

In [0]:
from sklearn.preprocessing import StandardScaler

df_osnovni[num_columns] = StandardScaler().fit_transform(df_osnovni[num_columns])
df_osnovni.head()

,Country,Happiness.Rank,Happiness.Score,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,Region,Happiness Rank 2016,Happiness Score 2016,Happiness Rank 2015,Happiness Score 2015,Air pollution (Micrograms per cubic metre),Dwellings without basic facilities (Percentage),Educational attainment (Percentage),Employees working very long hours (Percentage),Employment rate (Percentage),Feeling safe walking alone at night (Percentage),Homicide rate (Ratio),Household net adjusted disposable income (US Dollar),Household net financial wealth (US Dollar),Housing expenditure (Percentage),Labour market insecurity (Percentage),Life expectancy (Years),Life satisfaction (Average score),Long-term unemployment rate (Percentage),Personal earnings (US Dollar),Quality of support network (Percentage),Rooms per person (Ratio),Self-reported health (Percentage),Stakeholder engagement for developing regulations (Average score),Student skills (Average score),Time devoted to leisure and personal care (Hours),Voter turnout (Percentage),Water quality (Percentage),Years in education (Years),RankDiff2017,RankDiff2016
0,Afghanistan,1.465876,-1.440368,-1.447685,-2.208294,-1.632718,-2.107119,0.503613,-0.636605,0.627821,Southern Asia,1.712290,-1.838109,1.661870,-1.626841,6.417742e-16,0.0,0.0,0.0,0.0,2.294792e-15,0.0,9.658567e-16,-4.005354e-16,0.0,-3.700880e-16,0.0,-2.437925e-15,0.0,-1.087467e-15,6.591081e-15,-1.001841e-15,2.166626e-15,-1.359171e-15,0.0,1.001817e-14,0.0,0.0,0.0,-0.859323,0.108042
1,Albania,0.721304,-0.655562,0.028481,-1.400603,0.792224,-0.190008,-0.353144,-0.855376,-0.751547,Central and Eastern Europe,0.685178,-0.660990,0.350575,-0.376491,6.417742e-16,0.0,0.0,0.0,0.0,2.294792e-15,0.0,9.658567e-16,-4.005354e-16,0.0,-3.700880e-16,0.0,-2.437925e-15,0.0,-1.087467e-15,6.591081e-15,-1.001841e-15,2.166626e-15,-1.359171e-15,0.0,1.001817e-14,0.0,0.0,0.0,0.070130,1.036494
2,Algeria,-0.581697,0.478252,0.265952,-0.155182,0.291849,-1.221700,-1.375105,0.236055,1.498444,Middle East and Northern Africa,-0.935377,0.884262,-0.259855,0.207126,6.417742e-16,0.0,0.0,0.0,0.0,2.294792e-15,0.0,9.658567e-16,-4.005354e-16,0.0,-3.700880e-16,0.0,-2.437925e-15,0.0,-1.087467e-15,6.591081e-15,-1.001841e-15,2.166626e-15,-1.359171e-15,0.0,1.001817e-14,0.0,0.0,0.0,1.142576,-2.105960
3,Angola,1.442608,-1.439444,-0.313469,-0.307184,-2.209325,-2.846471,-1.154326,-0.548633,-0.492449,Sub-Saharan Africa,1.415568,-1.378169,1.300133,-1.213069,6.417742e-16,0.0,0.0,0.0,0.0,2.294792e-15,0.0,9.658567e-16,-4.005354e-16,0.0,-3.700880e-16,0.0,-2.437925e-15,0.0,-1.087467e-15,6.591081e-15,-1.001841e-15,2.166626e-15,-1.359171e-15,0.0,1.001817e-14,0.0,0.0,0.0,-0.001366,0.322300
4,Argentina,-1.256465,1.149492,0.497861,0.914629,0.633520,0.596980,-1.064971,-0.651173,1.595368,Latin America and Caribbean,-1.209274,1.152408,-1.118979,1.082552,6.417742e-16,0.0,0.0,0.0,0.0,2.294792e-15,0.0,9.658567e-16,-4.005354e-16,0.0,-3.700880e-16,0.0,-2.437925e-15,0.0,-1.087467e-15,6.591081e-15,-1.001841e-15,2.166626e-15,-1.359171e-15,0.0,1.001817e-14,0.0,0.0,0.0,-0.072863,-0.249055


###### Dolocanje vhodnih in izhodnih atributov

In [0]:
df = df_osnovni[['Happiness.Score', 'Economy..GDP.per.Capita.', 'Health..Life.Expectancy.', 'Freedom', 'Trust..Government.Corruption.']]

input_col = ['Economy..GDP.per.Capita.', 'Health..Life.Expectancy.', 'Freedom', 'Trust..Government.Corruption.']
output_col = 'Happiness.Score'

df_r_input = df[input_col]
df_r_output = df[[output_col]]
print(df.shape, df_r_input.shape, df_r_output.shape)

(168, 5) (168, 4) (168, 1)


#### Regresija

###### Izvajanje mreznega iskanje pravih vrednosti za parametre, ter regresije z navzkrizno validacijo s 5 rezi

In [0]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
import sklearn

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    #params used
    # max_features => max number of features considered for splitting a node
    # n_estimators => number of trees in forest
    params_rfr = { 'n_estimators': [5, 10, 50, 100, 500, 1000, 2500, 5000],
                  'max_features': ['auto', 'sqrt', 'log2']}

    gs_cv = GridSearchCV(RandomForestRegressor(), params_rfr, cv=5, scoring='neg_mean_squared_error')

    gs_cv.fit(df_r_input, df_r_output)

    print('Best parameters values:', gs_cv.best_params_)
    print('Best score:', gs_cv.best_score_)
    print('Best estimator:', gs_cv.best_estimator_)
#sklearn.metrics.SCORERS.keys()

Best parameters values: {'max_features': 'sqrt', 'n_estimators': 50}
Best score: -0.25514687035516725
Best estimator: RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)


#### Klasifikacija

###### Dodajanje regije v predhodno zajeti df s potrebni atributi

In [0]:
df['Region'] = df_osnovni['Region']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


###### Dodajanje stolpca kontinent

In [0]:
df['continent'] = pd.Series(list(range(len(df))))
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Happiness.Score,Economy..GDP.per.Capita.,Health..Life.Expectancy.,Freedom,Trust..Government.Corruption.,Region,continent
0,-1.440368,-1.447685,-1.632718,-2.107119,-0.636605,Southern Asia,0
1,-0.655562,0.028481,0.792224,-0.190008,-0.855376,Central and Eastern Europe,1
2,0.478252,0.265952,0.291849,-1.221700,0.236055,Middle East and Northern Africa,2
3,-1.439444,-0.313469,-2.209325,-2.846471,-0.548633,Sub-Saharan Africa,3
4,1.149492,0.497861,0.633520,0.596980,-0.651173,Latin America and Caribbean,4


###### Dodajanje vrednosti za stolpec kontinent glede na regijo

In [0]:
df.loc[df['Region'] == 'Australia and New Zealand', 'continent'] = 'Other'
df.loc[df['Region'].isin(['Central and Eastern Europe', 'Western Europe']), 'continent'] = 'Europe'
df.loc[df['Region'].isin(['Eastern Asia', 'Southeastern Asia', 'Southern Asia']), 'continent'] = 'Asia'
df.loc[df['Region'].isin(['Latin America and Caribbean', 'North America']), 'continent'] = 'Americas'
df.loc[df['Region'].isin(['Middle East and Northern Africa', 'Sub-Saharan Africa']), 'continent'] = 'Africa'
df.head(20)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Happiness.Score,Economy..GDP.per.Capita.,Health..Life.Expectancy.,Freedom,Trust..Government.Corruption.,Region,continent
0,-1.440368,-1.447685,-1.632718,-2.107119,-0.636605,Southern Asia,Asia
1,-0.655562,0.028481,0.792224,-0.190008,-0.855376,Central and Eastern Europe,Europe
2,0.478252,0.265952,0.291849,-1.221700,0.236055,Middle East and Northern Africa,Africa
3,-1.439444,-0.313469,-2.209325,-2.846471,-0.548633,Sub-Saharan Africa,Africa
4,1.149492,0.497861,0.633520,0.596980,-0.651173,Latin America and Caribbean,Americas
5,-0.903006,-0.208801,0.379697,-1.465640,-0.990889,Central and Eastern Europe,Europe
6,1.781953,1.240317,1.288863,1.342660,1.829433,Australia and New Zealand,Other
7,1.525276,1.246975,1.163044,1.107016,1.006242,Western Europe,Europe
8,-0.110814,0.419192,-0.046546,-0.074020,0.594531,Central and Eastern Europe,Europe
9,0.676762,1.250239,0.448464,0.891022,1.375920,Middle East and Northern Africa,Africa


###### Izkljucitev regije iz potrebnih atributov

In [0]:
df.drop('Region', axis=1, inplace=True)
df.columns
df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Happiness.Score,Economy..GDP.per.Capita.,Health..Life.Expectancy.,Freedom,Trust..Government.Corruption.,continent
0,-1.440368,-1.447685,-1.632718,-2.107119,-0.636605,Asia
1,-0.655562,0.028481,0.792224,-0.190008,-0.855376,Europe
2,0.478252,0.265952,0.291849,-1.221700,0.236055,Africa
3,-1.439444,-0.313469,-2.209325,-2.846471,-0.548633,Africa
4,1.149492,0.497861,0.633520,0.596980,-0.651173,Americas


In [0]:
#standardiziranje nominalnega atributa.... a je sploh potrebno ce je izhondi atribut??
from sklearn.preprocessing import LabelEncoder

df['continent'] = LabelEncoder().fit_transform(df['continent'])
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Happiness.Score,Economy..GDP.per.Capita.,Health..Life.Expectancy.,Freedom,Trust..Government.Corruption.,continent
0,-1.440368,-1.447685,-1.632718,-2.107119,-0.636605,2
1,-0.655562,0.028481,0.792224,-0.190008,-0.855376,3
2,0.478252,0.265952,0.291849,-1.221700,0.236055,0
3,-1.439444,-0.313469,-2.209325,-2.846471,-0.548633,0
4,1.149492,0.497861,0.633520,0.596980,-0.651173,1


###### Dolocanje vhodnih in izhodnih atributov

In [0]:
c_input_col = ['Happiness.Score', 'Economy..GDP.per.Capita.', 'Health..Life.Expectancy.', 'Freedom', 'Trust..Government.Corruption.']
c_output_col = 'continent'

df_c_input = df[c_input_col]
df_c_output = df[[c_output_col]]
print(df.shape, df_c_input.shape, df_c_output.shape)

(168, 6) (168, 5) (168, 1)


###### Izvajanje nakljucnega iskanja pravih vrednosti parametrov v 1000 iteracij, ter klasifikacije z navzkrizno validacijo s 5 rezi

In [0]:
from random import randint
import numpy as np
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    #params used
    # n_estimators => number of trees in the forest
    # criterion => measures the quality of a split
    # max_features => max number of features considered for splitting a node
    params_rfc = { 'n_estimators': sp_randint(5, 5000), 
                  'criterion': ['gini', 'entropy'],
                  'max_features': ['auto', 'sqrt', 'log2']}

    rs_cv = RandomizedSearchCV(RandomForestClassifier(), params_rfc, cv=5, n_iter=1000, scoring='accuracy')

    rs_cv.fit(df_c_input, df_c_output)
    print('Best parameters values:', rs_cv.best_params_)
    print('Best score:', rs_cv.best_score_)
    print('Best estimator:', rs_cv.best_estimator_)